In [65]:
#!pip install python-terrier
import pyterrier as pt
pt.init()

RuntimeError: pt.init() has already been called. Check pt.started() before calling pt.init()

In [19]:
# Load the dataset

import pandas as pd

dataset = pt.get_dataset("trec-deep-learning-passages")
train_topics = dataset.get_topics("train")
train_qrels = dataset.get_qrels("train")
train_topics = train_topics.sort_values(by='qid', ascending=True)[3:]
train_qrels = train_qrels.sort_values(by='qid', ascending=True)[3:]
# A simple join operation on the qids, as we have much more queries than qrels, we want to only use those
# queries with a qrel for optimal training performances. 
temp = pd.merge(train_topics, train_qrels, left_on='qid', right_on='qid')
# Make the new train_topics dataframe with only those queries that have a qrel
train_topics_100 = pd.read_csv("train_topics_100.csv", header=0)
print(train_topics_100 )

22:22:44.318 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8
        qid                                              query
0   1000005                    where is westminster california
1   1000007                            where is westminster ma
2   1000008                            where is westminster md
3   1000009                 where is westmont illinois located
4   1000010                              where is westmoreland
..      ...                                                ...
95  1000154                        where is william key buried
96  1000155                         where is william levy from
97  1000156                    where is william whipple buried
98  1000160                           where is williamsburg ky
99  1000162  where is williamsburg located on the map back ...

[100 rows x 2 columns]


In [20]:
train_qrel_100 = pd.concat([temp['qid'], temp['docno'], temp['label']], axis=1, keys=['qid', 'docno', 'label'])
train_qrel_100.head(100)

,qid,docno,label
0,1000005,3617173,1
1,1000007,5939245,1
2,1000008,4630854,1
3,1000009,1728286,1
4,1000010,4186186,1
...,...,...,...
95,100015,3035531,1
96,1000150,3598087,1
97,1000153,3606427,1
98,1000154,4609289,1


In [ ]:
#print(dataset.get_corpus())

In [35]:
import pandas as pd
file = 'C:/Users/vrnsr/Downloads/collection/collection.tsv'
documents = []
with open(file ,encoding="utf8") as f:
  for line in f:
    documents.append(line.split('\t'))

documents = pd.DataFrame(documents, columns=["doc_id","passage"])

print(documents.head())
documents['passage'] = documents['passage'].apply(lambda x: x.lower())
print(documents.head())

  doc_id                                            passage
0      0  The presence of communication amid scientific ...
1      1  The Manhattan Project and its atomic bomb help...
2      2  Essay on The Manhattan Project - The Manhattan...
3      3  The Manhattan Project was the name for a proje...
4      4  versions of each volume as well as complementa...
  doc_id                                            passage
0      0  the presence of communication amid scientific ...
1      1  the manhattan project and its atomic bomb help...
2      2  essay on the manhattan project - the manhattan...
3      3  the manhattan project was the name for a proje...
4      4  versions of each volume as well as complementa...


In [8]:
!pip install gensim

In [16]:
#Note to self - check length of the vector if it is 300 for both fasttext and Glove
import numpy as np
#load model accordingly and load embeddings
from gensim.models import KeyedVectors
model =  KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')

In [17]:
# Function returning vector reperesentation of a document
def get_embedding(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for word in doc_tokens:
            #if token exists in the trained model's vocabulary then fetch its vector
            try:
                embeddings.append(model.get_vector(word))
            #if it is an out-of-vocabulary word, generate a random vector of len 300
            except:
                embeddings.append(np.random.rand(300))

        #Note to self - understand why we are computing mean of these vectors        
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

In [36]:
'''import time
a = time.time()
#x1 = documents[:100000]
print(len(documents))
#x1['vector']= x1['passage'].progress_apply(lambda x :get_embedding(x.split()))
#print("time = ", time.time()-a)
documents['vector'] = documents['passage'].progress_apply(lambda x :get_embedding(x.split()))
#x1.progress_apply(lambda x: x**2)'''
#8,841,823

'import time\na = time.time()\n#x1 = documents[:100000]\nprint(len(documents))\n#x1[\'vector\'] = x1[\'passage\'].progress_apply(lambda x :get_embedding(x.split()))\n#print("time = ", time.time()-a)\ndocuments[\'vector\'] = documents[\'passage\'].progress_apply(lambda x :get_embedding(x.split()))\n#x1.progress_apply(lambda x: x**2)'

In [64]:
#rain_topics_100['query'] = train_topics_100['query'].apply(lambda x :x.lower())
#train_topics_100['vector'] = train_topics_100['query'].apply(lambda x :get_embedding(x.split()))
#print(train_topics_100)
#train_topics_100.to_csv("train_topics_100_fasttext_vectors.csv", index=False)

In [37]:
documents.head()

,doc_id,passage
0,0,the presence of communication amid scientific ...
1,1,the manhattan project and its atomic bomb help...
2,2,essay on the manhattan project - the manhattan...
3,3,the manhattan project was the name for a proje...
4,4,versions of each volume as well as complementa...


In [18]:
#documents = documents.drop(['passage'], axis = 1)

In [40]:
#documents.to_csv("Documents_FasttextVectors_withoutPassage.csv", index=False)

In [28]:
print(documents.head())

  doc_id                                            passage
0      0  the presence of communication amid scientific ...
1      1  the manhattan project and its atomic bomb help...
2      2  essay on the manhattan project - the manhattan...
3      3  the manhattan project was the name for a proje...
4      4  versions of each volume as well as complementa...


In [38]:
import gc
import re
import pandas as pd
# Returns the number of
# objects it has collected
# and deallocated
collected = gc.collect()
train_topics_100 = pd.read_csv("train_topics_100_fasttext_vectors.csv", header = 0)
train_topics_100 = train_topics_100.drop(['query'],axis=1)
train_topics_100['vector'] = train_topics_100['vector'].map(lambda vec: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', vec).splitlines())))
print(train_topics_100)
#y1 = y1.merge(documents[:800000], how='cross')
y1 = 0

        qid                                             vector
0   1000005  [-0.00582499988, 0.0814249963, -0.0252250005, ...
1   1000007  [-0.034874998, 0.0872249976, -0.0623999983, -0...
2   1000008  [0.000274999999, 0.087349996, -0.0505999997, 0...
3   1000009  [0.16909066, 0.12716744, 0.09672532, 0.1944896...
4   1000010  [0.0204666667, 0.0631666705, -0.0222333353, -0...
..      ...                                                ...
95  1000154  [0.0368000045, 0.0254399981, -0.0241400003, -0...
96  1000155  [0.00324000092, -0.00404000143, 0.0218599997, ...
97  1000156  [0.072695575, 0.118829952, 0.0220951142, 0.162...
98  1000160  [-0.00187500101, 0.0962999985, -0.0557250008, ...
99  1000162  [0.019955555, 0.0489555523, -0.0441444442, -0....

[100 rows x 2 columns]


In [22]:
#documents['vector'] = documents['vector'].map(lambda vec: ast.literal_eval(','.join(re.sub(r'(?<=\d)(\s+)(?=-?\d)', ',', vec).splitlines())))

In [39]:
from numpy import dot
from numpy.linalg import norm

import ast
model = 0
collected = gc.collect()
def cosine(x,y):
    return dot(x,y)/norm(x)/norm(y)

#print(list(y1))
#y1.progress_apply(lambda x: cosine(x['vector_x'], x['vector_y']), axis=1)

In [47]:
from tqdm.notebook import tqdm
tqdm.pandas()
batch = 100000
start = 8300000
for i in range(start,len(documents),batch):
    up = min(i+batch, len(documents))
    print(i, up)
    name = "doc_vec_fastext" + str(i) + "-" + str(up)
    doc_subset = documents[i:up]
    #print(type(doc_subset), doc_subset[:10])
    doc_subset['vector'] = doc_subset['passage'].progress_apply(lambda x :get_embedding(x.split()))
    doc_subset = doc_subset.drop(['passage'],axis=1)
    y1 = train_topics_100.merge(doc_subset, how='cross')
    doc_subset = 0
    #print(y1)
    y1['cosine'] = y1.progress_apply(lambda x: cosine(x['vector_x'], x['vector_y']), axis=1)
    y1 = y1.drop(['vector_x','vector_y'], axis=1)
    y1.to_csv(name, index=False)

8300000 8400000


  0%|          | 0/100000 [00:00<?, ?it/s]

<ipython-input-47-5b7b2874b557>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_subset['vector'] = doc_subset['passage'].progress_apply(lambda x :get_embedding(x.split()))


  0%|          | 0/10000000 [00:00<?, ?it/s]

8400000 8500000


...

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/10000000 [00:00<?, ?it/s]

8800000 8841823


  0%|          | 0/41823 [00:00<?, ?it/s]

  0%|          | 0/4182300 [00:00<?, ?it/s]